In [1]:
import os
import numpy as np
import tensorflow as tf
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


## 1. Data Pre-processing

### 1.1 Read sequence from file

In [2]:
def vectorizeSeq(fileName):
    file = SeqIO.parse(open(fileName),'fasta')
    oneHot = []
    numLable = []
    seq_name = []
    seq_gene = []
    i = 0
    for seq in file:
        seq_name.append(seq.id)
        seq_gene.append(str(seq.seq))
    
        #One-hot Encoding
        oneHotDict = {'A': np.array([1,0,0,0]),
                      'C': np.array([0,1,0,0]),
                      'G': np.array([0,0,1,0]),
                      'T': np.array([0,0,0,1]),
                      'N': np.array([0,0,0,0])} # may need change

        oneHot.append(np.array(list(map(lambda letter: oneHotDict[letter], seq_gene[i])))
                        )
        numLableDict = {'A': np.array([0]),
                        'C': np.array([1]),
                        'G': np.array([2]),
                        'T': np.array([3]),
                        'N': np.array([4])} # may need change

        numLable.append(np.array(list(map(lambda letter: numLableDict[letter], seq_gene[i])))
                        )
        i+=1
    return np.array(oneHot), np.array(numLable), seq_gene

In [3]:
negative_onehot,nagative_num,nagative_seq = vectorizeSeq("promoter_800_900.fa")
positive_onehot,positive_num,positive_seq = vectorizeSeq("promoter_-50_50.fa")

In [4]:
#print(promoter[0])
#print(promoter_onehot[0])
#print(promoter_num[0])

print(positive_onehot.shape)
print(negative_onehot.shape)

(16455, 101, 4)
(16455, 101, 4)


### 1.2 Creating Labels

In [5]:
def lableGenerator(positive, nagative):
    y = np.array([1, 0])
    y = np.repeat(y, [positive.shape[0], nagative.shape[0]])
    y = to_categorical(y)
    return y

### 1.3 Creating dataset

In [6]:
y = lableGenerator(positive_onehot, negative_onehot)

In [7]:
def PromoterDataset(positive, negative):
    return np.concatenate((positive, negative), axis = 0)

In [8]:
x = PromoterDataset(positive_onehot, negative_onehot)

We will split the whole dataset into:
* 60% for training
* 20% for validation
* 20% for testing

In [9]:
indices100 = np.arange(x.shape[0])
X100_train, X100_test, y100_train, y100_test, idx100_train, idx100_test = train_test_split(x, y, indices100, test_size = 0.2, random_state = 42)

## 2. Build Model

In [10]:
from keras.models import Model
from keras.models import Sequential
from keras.models import model_from_yaml
from keras.layers.convolutional import Conv1D # Keras 2 syntax
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.initializers import Constant
from keras.callbacks import EarlyStopping

#from keras.callbacks import TensorBoard

from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import time

In [11]:
def CNN1(length, dimensions, nb_filters, kernel_size, pool_size):
    # Initialize network
    classifier = Sequential()
    
    # Layers 1 and 2: Add convolutional layers
    classifier.add(Conv1D(filters=nb_filters, kernel_size=kernel_size, input_shape=(length, dimensions), activation="relu"))
    
    # Layer 3: Max-Pool // may change to average pool
    classifier.add(MaxPooling1D(pool_size = pool_size))
    
    # Layer 7: Flatten to be fed into fully-connected layers
    classifier.add(Flatten())
    
    # Layers 8: fully connected layer with 256 nodes
    classifier.add(Dense(units = 128, activation = 'relu'))
    
    # Layer 11: final layer for binary prediction using softmax activation function
    classifier.add(Dense(units = 2, activation='softmax'))
    
    print(classifier.summary())
    # Return compiled network
    return classifier

#classifier = CNN1(251, 4, 300, 21, 231)

In [12]:
classifier = CNN1(101, 4, 300, 21, 81)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 81, 300)           25500     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 300)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               38528     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 64,286
Trainable params: 64,286
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # summarize history for training accuracy, validation accuracy, and loss
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    
    # Set titles/labels, labels, etc.
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

## 3. Training

### 3.1 Early Stopping

In [14]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')

callbacks_list = [earlystop]

### 3.2 The Adama optimizer and binary cross-entropy

In [15]:
# Compile neural network
classifier.compile(loss='binary_crossentropy', # Cross-entropy
                   optimizer='adam',
                   metrics=['accuracy']) # Accuracy performance metric

### 3.3 Run on GPU

In [ ]:
# train the model
start = time.time()

model_info = classifier.fit(X100_train, y100_train, verbose=1, \
                            validation_steps = 0, \
                            validation_split = 0.25, \
                            batch_size = 16, \
                            callbacks = callbacks_list, \
                            epochs = 30)

end = time.time()

Train on 19746 samples, validate on 6582 samples
Epoch 1/30


### 3.4 Plotting

In [ ]:
plot_model_history(model_info)

In [ ]:
# compute test accuracy
print("Accuracy on test data is: {0}".format(classifier.evaluate(X100_test, y100_test)))

## 4. Model Evaluation

### 4.1 Make predictions

In [ ]:
y_pred = classifier.predict(X100_test)
rounded = [round(val[1]) for val in y_pred]

In [ ]:
classifier.evaluate(X100_test, y100_test)

### 4.2 Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y100_test[:,0], rounded))

### 4.3 Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y100_test[:,1], rounded))